In [18]:
import sys
import logging
from uuid import UUID

import emission.core.get_database as edb
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import datetime as pydt
import emission.storage.timeseries.abstract_timeseries as esta

logging.getLogger().setLevel(logging.DEBUG)
logging.debug(sys.path)

iphone_ids = [UUID("079e0f1a-c440-3d7c-b0e7-de160f748e35"), UUID("c76a0487-7e5a-3b17-a449-47be666b36f6"), 
              UUID("c528bcd2-a88b-3e82-be62-ef4f2396967a")]
android_ids = [UUID("e471711e-bd14-3dbe-80b6-9c7d92ecc296"), UUID("fd7b4c2e-2c8b-3bfa-94f0-d1e3ecbd5fb7"),
               UUID("86842c35-da28-32ed-a90e-2da6663c5c73")]

In [19]:
list(edb.get_timeseries_db().find({"metadata.key": "config/sensor_config", "data.battery_status": {"$exists": True}}).sort("metadata.write_ts").limit(10))

[]

In [44]:
#type: list of emission.storage.timeseries.builtin_timeseries.BuiltinTimeSeries
iphone_ts = map(lambda u: esta.TimeSeries.get_time_series(u), iphone_ids)
android_ts = map(lambda u: esta.TimeSeries.get_time_series(u), android_ids)

#BuiltinTimeSeries.get_data_df(key, time_query = None, geo_query = None, extra_query_list=None) 
#type: list of DataFrame objects (regarding state info)
_iphone_state_df_list = map(lambda t:t.get_data_df("statemachine/transition"), iphone_ts)
_android_state_df_list = map(lambda t:t.get_data_df("statemachine/transition"), android_ts)

_iphone_battery_df_list = map(lambda t:t.get_data_df("background/battery"), iphone_ts)
_android_battery_df_list = map(lambda t:t.get_data_df("background/battery"), android_ts)

_iphone_config_df_list = map(lambda t:t.get_data_df("config/sensor_config"), iphone_ts)
_android_config_df_list = map(lambda t:t.get_data_df("config/sensor_config"), android_ts)

ts_to_dt = lambda t: pydt.datetime.fromtimestamp(t)

i_config_df_list=[]
for df in _iphone_config_df_list:
    df['local_dt'] = df['metadata_write_ts'].apply(ts_to_dt)
    df['ts'] = df['metadata_write_ts']
    i_config_df_list.append(df)

a_config_df_list=[]
for df in _android_config_df_list:
    df['local_dt'] = df['metadata_write_ts'].apply(ts_to_dt)
    df['ts'] = df['metadata_write_ts'] #<class 'pandas.core.series.Series'>
    a_config_df_list.append(df)

In [45]:
def local_dt_range(t):
    #time range specification
    dt1 = pydt.datetime(2016, 6, 24, 22, 0) #datetime.datetime(2016, 6, 24, 22, 0)
    dt2 = pydt.datetime(2016, 6, 26, 23, 0)
    dic = {}
    if (type(t) == type(dic)):
        y = t.get('year')
        m = t.get('month')
        d = t.get('day')
        h = t.get('hour')
        mi = t.get('minute')
        dt = pydt.datetime(y, m, d, h, mi)
        if dt1<=dt and dt<=dt2:
            return True
        else:
            return False
        
    ts = pd.Timestamp(pydt.datetime(2012, 5, 1))
    if (type(t) == type(ts)):
        dt = t.to_datetime()
        if dt1<=dt and dt<=dt2:
            return True
        else:
            return False
    return False

In [46]:
#query info from a specified time range
#columns:_id|curr_state|fmt_time|local_dt|metadata_write_ts|transition|ts#
#testified: pydt.datetime.fromtimestamp(ts/meta_ts) = fmt_time/local_dt#
iphone_state_df_list = map(lambda t: t[t.local_dt.apply(local_dt_range)], _iphone_state_df_list)
android_state_df_list = map(lambda t: t[t.local_dt.apply(local_dt_range)], _android_state_df_list)

#columns: _id|battery_level_pct|battery_status|fmt_time|local_dt|metadata_write_ts|ts#
#testified: pydt.datetime.fromtimestamp(ts/meta_ts) = fmt_time/local_dt#
iphone_battery_df_list = map(lambda t: t[t.local_dt.apply(local_dt_range)], _iphone_battery_df_list)
android_battery_df_list = map(lambda t: t[t.local_dt.apply(local_dt_range)], _android_battery_df_list)

#_id|accuarcy|accuracy_threshold|android_geofence_responsiveness|filter_distance|filter_time|geofence_radius|
#ios_use_remote_push_for_sync|ios_use_visit_noti_for_detection|is_duty_cycling|metadata_write_ts|simulate_user_inter|
#trip_end_stationary_mins|
#"new_columns": local_dt|ts|
iphone_config_df_list = map(lambda t: t[t.local_dt.apply(local_dt_range)], i_config_df_list)
android_config_df_list = map(lambda t: t[t.local_dt.apply(local_dt_range)], a_config_df_list)

In [47]:
ios1_s = iphone_state_df_list[0]
ios1_s[0:1]
ios1_b = iphone_battery_df_list[0]
#ios1_b
ios1_c = iphone_config_df_list[0]
#ios1_c

In [51]:
ts_to_dt = lambda t: pydt.datetime.fromtimestamp(t)
#ios1_b.loc[:,'dt'] = ios1_b['metadata_write_ts'].apply(ts_to_dt)
ios1_c[['metadata_write_ts','local_dt','ts']]

,metadata_write_ts,local_dt,ts
4,1.466831e+09,2016-06-24 22:01:39.626626,1.466831e+09
5,1.466831e+09,2016-06-24 22:07:36.299538,1.466831e+09
6,1.466831e+09,2016-06-24 22:07:36.855116,1.466831e+09
7,1.466833e+09,2016-06-24 22:35:24.139247,1.466833e+09
8,1.466834e+09,2016-06-24 22:46:34.723293,1.466834e+09
9,1.466994e+09,2016-06-26 19:18:16.027431,1.466994e+09
10,1.466994e+09,2016-06-26 19:25:11.911947,1.466994e+09
11,1.466994e+09,2016-06-26 19:25:16.285784,1.466994e+09
